In [ ]:
#options used later to simplify arguments
# Set up headless browser which stops any GUI from apearing
options = Options()
options.add_argument('--headless')
#stops Graphics processing unit from rendering the web content
options.add_argument('--disable-gpu')

#sets up remote control interface to instruct the behavior of web browsers
#basically sets up functionallity to scrape web
driver = webdriver.Chrome(options=options)
def getDataFrame(year, statArr):
    all_stat_dfs = []
    rowsArr = []
    headersArr = []
    for stat in statArr:
        url = f'https://www.nfl.com/stats/team-stats/offense/{stat}/{year}/reg/all'
        driver.get(url)

        time.sleep(2)
        #reads through page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find('table')
        #strips *table header* elements
        headers = [th.text.strip() for th in table.find_all('th')]
        headersArr.append(headers)
        #strips all rows for each team
        rows = []
        for row in table.find_all('tr')[1:]:
            #td holds info, tr is the stucture
            cols = [td.text.strip() for td in row.find_all('td')]
            if cols:
                rows.append(cols)
        df_stat = pd.DataFrame(rows, columns=headers)
        #makes lard dataframe that will soon be condensed into one
        all_stat_dfs.append(df_stat)
    #should always be the same
    print(len(headers), len(rows[0]))
    #getting the condesed dataframe and return it 
    df_merged = reduce(lambda left, right: pd.merge(left, right, on='Team'), all_stat_dfs)
    return df_merged

offensiveStats2020 = getDataFrame(2020, offensiveStatsArr)
defensiveStats2020 = getDataFrame(2020, defensiveStatsArr)
STStats2020 = getDataFrame(2020, STStatsArr)

